In [1]:
CELL_TYPE = 'pDC'
N_TRIALS: int = 50
SPLIT_IDX = 0

In [2]:
import os
import sys
from pyprojroot.here import here

import pandas as pd
import anndata as ad
import numpy as np
import math
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import product

import optuna

import joblib
import pickle
import datetime

import collections

import xgboost
from sklearn.preprocessing import LabelEncoder

import scipy.sparse as ssp
import joblib

from dotenv import load_dotenv

In [3]:
assert load_dotenv()

# LOAD DATASET

In [5]:
adata_ct = ad.read_h5ad(here(f'03_downstream_analysis/08_gene_importance/COMBAT_focus_analysis/data/{CELL_TYPE}_COMBAT2022_spectraGenes_log1p.h5ad'))

In [6]:
adata_ct

AnnData object with n_obs × n_vars = 1837 × 935
    obs: 'studyID', 'libraryID', 'sampleID', 'chemistry', 'disease', 'sex', 'binned_age', 'Level1', 'Level2', 'batch', 'COVID_severity'
    var: 'hgnc_id', 'symbol', 'locus_group', 'HUGO_status', 'highly_variable'
    uns: 'log1p'

In [7]:
adata_ct.obs.studyID.unique(), adata_ct.obs.Level1.unique()

(['COMBAT2022']
 Categories (1, object): ['COMBAT2022'],
 ['pDC']
 Categories (1, object): ['pDC'])

### Generate traint - test - validation splits (5-fold cross validation setting)

In [8]:
N_SPLITS = 5

In [9]:
left_out_splits = [s[1] for s in StratifiedGroupKFold(n_splits=N_SPLITS).split(X = adata_ct.obs.index, y=adata_ct.obs.disease, groups=adata_ct.obs.batch)]

In [10]:
test_split_n = SPLIT_IDX
validation_split_n = (SPLIT_IDX + 1) % N_SPLITS

test_idxs = left_out_splits[test_split_n]
val_idxs = left_out_splits[validation_split_n]

train_idxs = list()

for idx in np.setdiff1d(range(N_SPLITS), np.array([test_split_n,validation_split_n])):
    train_idxs.append(left_out_splits[idx])
train_idxs = np.concatenate(train_idxs)

In [11]:
train_idxs.shape, test_idxs.shape, val_idxs.shape, train_idxs.shape[0]+val_idxs.shape[0]+test_idxs.shape[0]

((1037,), (424,), (376,), 1837)

In [12]:
X_train = adata_ct[train_idxs,:].X
X_test = adata_ct[test_idxs, :].X
X_val = adata_ct[val_idxs, :].X
X_train.shape, X_test.shape, X_val.shape

((1037, 935), (424, 935), (376, 935))

In [13]:
y_train = adata_ct.obs.iloc[train_idxs].disease.values.astype(str)
y_test = adata_ct.obs.iloc[test_idxs].disease.values.astype(str)
y_val = adata_ct.obs.iloc[val_idxs].disease.values.astype(str)
y_train.shape, y_test.shape, y_val.shape 

((1037,), (424,), (376,))

In [14]:
lenc = LabelEncoder()
y_train_enc = lenc.fit_transform(y_train)
y_val_enc = lenc.transform(y_val)
y_test_enc = lenc.transform(y_test)

### GENERATE F1 

In [15]:
def custom_f1_score(y_true, y_pred):
    return -f1_score(y_true, y_pred.argmax(1), average='weighted')

In [16]:
eval_metric=custom_f1_score
eval_metric_name='custom_f1_score'

def objective(trial):
    params = {
        'sampling_method': 'gradient_based',
        'n_estimators': 1500,
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 250),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0.1, 1.0),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 5e-1, log=True),
    }
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, f'validation_0-{eval_metric_name}')
    xgb = xgboost.XGBClassifier(
        device='gpu',
        eval_metric=eval_metric,
        early_stopping_rounds=20,
        callbacks=[pruning_callback],
        **params
    )
    xgb.fit(
        X_train, 
        y_train_enc, 
        verbose=0,
        eval_set=[(X_val, y_val_enc)],
    )
    trial.set_user_attr('best_iteration', xgb.best_iteration)

    return xgb.best_score

In [17]:
sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective, n_trials=N_TRIALS, gc_after_trial=True)

[I 2025-04-04 15:36:52,664] A new study created in memory with name: no-name-a2c3c6f5-c797-4271-99a2-a184fca99852
[I 2025-04-04 15:36:53,534] Trial 0 finished with value: -0.531065 and parameters: {'max_depth': 9, 'min_child_weight': 238, 'subsample': 0.7587945476302645, 'colsample_bynode': 0.6387926357773329, 'learning_rate': 0.0026368755339723046}. Best is trial 0 with value: -0.531065.
[I 2025-04-04 15:36:54,217] Trial 1 finished with value: -0.690511 and parameters: {'max_depth': 5, 'min_child_weight': 15, 'subsample': 0.8795585311974417, 'colsample_bynode': 0.6410035105688879, 'learning_rate': 0.08148293210105287}. Best is trial 1 with value: -0.690511.
[I 2025-04-04 15:36:54,394] Trial 2 finished with value: -0.531065 and parameters: {'max_depth': 3, 'min_child_weight': 243, 'subsample': 0.8491983767203796, 'colsample_bynode': 0.29110519961044856, 'learning_rate': 0.003095566460242371}. Best is trial 1 with value: -0.690511.
[I 2025-04-04 15:36:54,688] Trial 3 finished with value

In [18]:
joblib.dump(study, here(f'03_downstream_analysis/08_gene_importance/COMBAT_focus_analysis/results/study/{SPLIT_IDX}_{CELL_TYPE}_xgboost.pkl'))

['/scratch_isilon/groups/singlecell/shared/projects/Inflammation-PBMCs-Atlas/03_downstream_analysis/08_gene_importance/COMBAT_focus_analysis/results/study/0_pDC_xgboost.pkl']

In [19]:
n_estimators = int(study.best_trial.user_attrs['best_iteration']*1.2)
xgb = xgboost.XGBClassifier(
        device='gpu',
        eval_metric=eval_metric,
        n_estimators=n_estimators,
        **study.best_trial.params
    )
xgb.fit(
    ssp.vstack((X_train, X_val)), 
    np.concatenate((y_train_enc, y_val_enc)), 
    eval_set=[(X_test, y_test_enc)],
    verbose=1,
)

[0]	validation_0-mlogloss:1.26009	validation_0-custom_f1_score:-0.29479
[1]	validation_0-mlogloss:1.15357	validation_0-custom_f1_score:-0.29479
[2]	validation_0-mlogloss:1.07931	validation_0-custom_f1_score:-0.29479
[3]	validation_0-mlogloss:0.99664	validation_0-custom_f1_score:-0.62880
[4]	validation_0-mlogloss:0.94824	validation_0-custom_f1_score:-0.60855
[5]	validation_0-mlogloss:0.90540	validation_0-custom_f1_score:-0.66008
[6]	validation_0-mlogloss:0.88836	validation_0-custom_f1_score:-0.64907
[7]	validation_0-mlogloss:0.87025	validation_0-custom_f1_score:-0.70176
[8]	validation_0-mlogloss:0.83486	validation_0-custom_f1_score:-0.72685
[9]	validation_0-mlogloss:0.83312	validation_0-custom_f1_score:-0.71766
[10]	validation_0-mlogloss:0.82193	validation_0-custom_f1_score:-0.72455
[11]	validation_0-mlogloss:0.81974	validation_0-custom_f1_score:-0.73102
[12]	validation_0-mlogloss:0.80884	validation_0-custom_f1_score:-0.73118
[13]	validation_0-mlogloss:0.79736	validation_0-custom_f1_sco

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=0.9539969835279999,
              colsample_bytree=None, device='gpu', early_stopping_rounds=None,
              enable_categorical=False,
              eval_metric=<function custom_f1_score at 0x70f9233ea2a0>,
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.4038423798071558, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=13, max_leaves=None,
              min_child_weight=43, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=38, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [20]:
joblib.dump(xgb, (here(f'03_downstream_analysis/08_gene_importance/COMBAT_focus_analysis/results/best_model/{SPLIT_IDX}_{CELL_TYPE}_xgb.json')))

['/scratch_isilon/groups/singlecell/shared/projects/Inflammation-PBMCs-Atlas/03_downstream_analysis/08_gene_importance/COMBAT_focus_analysis/results/best_model/0_pDC_xgb.json']

In [21]:
(pd.DataFrame(np.array((y_test, y_test_enc, xgb.predict(X_test))).T, columns=['y_true', 'y_true_code', 'y_pred'])
 .to_csv(here(f'03_downstream_analysis/08_gene_importance/COMBAT_focus_analysis/results/predictions/{SPLIT_IDX}_{CELL_TYPE}_pred_test.zip')))
(pd.DataFrame(np.array((y_train, y_train_enc, xgb.predict(X_train))).T, columns=['y_true', 'y_true_code', 'y_pred'])
 .to_csv(here(f'03_downstream_analysis/08_gene_importance/COMBAT_focus_analysis/results/predictions/{SPLIT_IDX}_{CELL_TYPE}_pred_train.zip')))

/scratch_isilon/groups/singlecell/shared/conda_env/xgboostGPU/lib/python3.12/site-packages/xgboost/core.py:160: UserWarning: [15:37:16] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1703076482591/work/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
